### Prepare a file for RoamDB for Partner

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'

file_partner_roamdb = 'partner_to_roamdb.csv'

In [2]:
# Из базы данных RoamDB выгружаем информацию по поддержке 4G

import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')

sql_srt=\
'''
SELECT TadigCodes.Value AS TADIG,Support4G FROM Tariffs
LEFT JOIN TadigCodes ON Tariffs.OperatorID = TadigCodes.OperatorID
WHERE
TariffListID =  21
AND Support4G = 1
'''
df_4g_partner = pd.read_sql_query(sql_srt, cnxn)

display(df_4g_partner.head())

cnxn.close()

,TADIG,Support4G
0,ALBVF,True
1,AUSOP,True
2,AUTPT,True
3,AZEAF,True
4,BELKO,True


In [3]:
# Загружаем новые тарифы Partner

df_partner = pd.read_csv(join(load_folder,'Partner_1_1_2019_for_tariffs.csv'),sep=';')
display(df_partner.head())

,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,ALBVF,27602.0,Vodafone AL,Albania,VF,0.0439,0.0,0.0062,0.0030,1,1.0
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.2000,0.0,0.0300,0.2000,60,1.0
2,AIACW,36584.0,AIACW AI,Anguilla,C&W Lime,0.1606,0.0,0.0128,0.0257,60,1.0
3,ATGCW,34492.0,ATGCW AG,Antigua And Barbuda,C&W Lime,0.1500,0.0,0.0120,0.0240,60,1.0
4,ARGTM,72207.0,TELE MOV AR,Argentina,Telefonica LATAM,0.0728,0.0,0.0240,0.0180,1,1.0


In [4]:
df = pd.merge(df_partner,df_4g_partner,how='left',on='TADIG')
df['IsDiscounted (Yes/No)'] = 'Yes'
df.rename({'TADIG':'TADIG CODE','MOC':'Mobile Origination','MTC':'Mobile Termination','MB':'DATA',
          'Voice_Increment_SEC':'Voice Increment','Data_Increment_KB':'Data Increment','Support4G':'Support4G(Yes/No)'},
         axis=1,inplace=True)
df.drop(['MCCMNC','PARTNER','COUNTRY','Operator'],axis=1,inplace=True)

df['Support4G(Yes/No)'].replace({True:'Yes',np.nan:'No'},inplace=True)

columns_ordered = ['TADIG CODE',
 'Mobile Origination',
 'Mobile Termination',
 'SMS',
 'DATA',
 'Voice Increment',
 'Data Increment',
 'IsDiscounted (Yes/No)',
 'Support4G(Yes/No)',]

df[columns_ordered].to_csv(join(load_folder,file_partner_roamdb),index=False)

df[columns_ordered].head()

,TADIG CODE,Mobile Origination,Mobile Termination,SMS,DATA,Voice Increment,Data Increment,IsDiscounted (Yes/No),Support4G(Yes/No)
0,ALBVF,0.0439,0.0,0.0062,0.0030,1,1.0,Yes,Yes
1,ANDMA,0.2000,0.0,0.0300,0.2000,60,1.0,Yes,No
2,AIACW,0.1606,0.0,0.0128,0.0257,60,1.0,Yes,No
3,ATGCW,0.1500,0.0,0.0120,0.0240,60,1.0,Yes,No
4,ARGTM,0.0728,0.0,0.0240,0.0180,1,1.0,Yes,No
